In [367]:
import os
import numpy as np
import pandas as pd

def find_r_installation():
    possible_paths = []

    # Add standard paths for Windows
    if os.getenv('ProgramFiles'):
        possible_paths.append(os.path.join(os.getenv('ProgramFiles'), 'R'))
    if os.getenv('ProgramFiles(x86)'):
        possible_paths.append(os.path.join(os.getenv('ProgramFiles(x86)'), 'R'))
    if os.getenv('LOCALAPPDATA'):
        possible_paths.append(os.path.join(os.getenv('LOCALAPPDATA'), 'Programs', 'R'))
    
    # Add standard path for Unix-like systems
    if os.getenv('HOME'):
        possible_paths.append(os.path.join(os.getenv('HOME'), 'R'))

    for path in possible_paths:
        if os.path.exists(path):
            for root, dirs, files in os.walk(path):
                if 'bin' in dirs:
                    if 'R.exe' in os.listdir(os.path.join(root, 'bin')):  # Check specifically for Windows
                        return os.path.join(root)
                    if 'R' in os.listdir(os.path.join(root, 'bin')):  # Check for Unix-like systems
                        return os.path.join(root)

    return None

r_path = find_r_installation()
if r_path:
    print(f"R is installed at: {r_path}")
else:
    print("R installation not found.")

os.environ['R_HOME'] = r_path

R is installed at: C:\Users\maceo.valente\AppData\Local\Programs\R\R-4.3.2


In [368]:
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import importr
from rpy2.robjects import r
from rpy2.robjects.vectors import StrVector, ListVector, IntVector, FloatVector

import ast

r.source("R_related/Rfunctions.R")

value,[RTYPES.CLOSXP]
visible,[RTYPES.LGLSXP]


In [369]:
inputs_path = "inputs.xlsx"

In [370]:
# Lire le fichier Excel
df = pd.read_excel(inputs_path, usecols=[1,2], skiprows=5)
df.columns = ['Valeur', 'Cle']


In [371]:
print(df)

                Valeur                   Cle
0                 2.25            PY_ENTRAXE
1                    6            PY_HAUTEUR
2                1.135            PY_RAMPANT
3                False         PY_TYPE_PANEL
4                 0.18      PY_LARGEUR_BANDE
5                    4          PY_NB_BANDES
6                   13  PY_ANGLE_ORIENTATION
7                    1          PY_TYPE_CULT
8                    1            PY_FINESSE
9                10000             PY_KEYKEY
10                 gre             PY_cepage
11                35.8            SOILS_calc
12                0.15            SOILS_norg
13                  22            SOILS_argi
14                  35         SOILS_profhum
15                  10        SOILS_CsurNsol
16                 8.2              SOILS_pH
17       [10, 40, 150]             SOILS_epc
18  [30,6 , 26 , 26,7]            SOILS_HCCF
19    [17.4,17.4,16.8]           SOILS_hminf
20     [1.5,1.45,1.45]             SOILS_DAF
21    [10.

In [372]:
# Initialiser un dictionnaire vide
result_dict = {}

# Parcourir les lignes du dataframe et remplir le dictionnaire
for index, row in df.iterrows():
    key = row['Cle']
    value = row['Valeur']
    
    # Si la valeur est une chaîne de caractères représentant une liste
    if isinstance(value, str) and value.startswith('[') and value.endswith(']'):
        value = ast.literal_eval(value)
    
    result_dict[key] = value
    print(index)
    print(key, 'value :', value)
print(result_dict)

0
PY_ENTRAXE value : 2.25
1
PY_HAUTEUR value : 6
2
PY_RAMPANT value : 1.135
3
PY_TYPE_PANEL value : False
4
PY_LARGEUR_BANDE value : 0.18
5
PY_NB_BANDES value : 4
6
PY_ANGLE_ORIENTATION value : 13
7
PY_TYPE_CULT value : 1
8
PY_FINESSE value : 1
9
PY_KEYKEY value : 10000
10
PY_cepage value : gre
11
SOILS_calc value : 35.8
12
SOILS_norg value : 0.15
13
SOILS_argi value : 22
14
SOILS_profhum value : 35
15
SOILS_CsurNsol value : 10
16
SOILS_pH value : 8.2
17
SOILS_epc value : [10, 40, 150]
18
SOILS_HCCF value : [30, 6, 26, 26, 7]
19
SOILS_hminf value : [17.4, 17.4, 16.8]
20
SOILS_DAF value : [1.5, 1.45, 1.45]
21
SOILS_cailloux value : [10.1, 10.4, 10.4]
22
INITS_Hinitf value : [7, 0, 0, 0, 0]
23
INITS_densinitial value : [4, 3, 2]
24
INITS_NO3initf value : [0, 0, 0, 0, 0]
25
INITS_NH4initf value : [0, 0, 0, 0, 0]
{'PY_ENTRAXE': 2.25, 'PY_HAUTEUR': 6, 'PY_RAMPANT': 1.135, 'PY_TYPE_PANEL': 'False', 'PY_LARGEUR_BANDE': 0.18, 'PY_NB_BANDES': 4, 'PY_ANGLE_ORIENTATION': 13, 'PY_TYPE_CULT': 1, 'P

In [373]:
(result_dict['SOILS_epc'])

[10, 40, 150]

In [374]:
r_settings = ro.globalenv['settings']

r_change_setting = ro.globalenv['change_setting']

#from rpy2.robjects.vectors import StrVector, ListVector, IntVector, FloatVector

In [375]:
pandas2ri.activate()

for key, value in result_dict.items():
    print(key,value)

    if key.startswith('PY_'):
        continue

    if key.startswith("SOILS_"):
        xmlfile = 'sols.xml'
    elif key.startswith("INITS_"):  
        xmlfile = 'VignePioGre_ini.xml'
    elif key.startswith("TECPL_"):
        xmlfile = 'VignePioGre_tec.xml'
    param_name = key[6:]  # Erase prefix

    print("PY ; xmlfile :", xmlfile)

    if isinstance(value, list):
        value_id = list(range(1, 6))
        value.extend([0] * (5 - len(value)))
        print("PY ; value_id :",value_id)
        print("PY ; value :",value)
        print("PY ; name :",param_name)
        r_change_setting(os.path.join('R_related','Rproject',xmlfile),
                            param_name = param_name,
                            val = value,
                            values_id=value_id)
    else:
        r_change_setting(os.path.join('R_related','Rproject',xmlfile),
                            param_name = param_name,
                            val = value)

PY_ENTRAXE 2.25
PY_HAUTEUR 6
PY_RAMPANT 1.135
PY_TYPE_PANEL False
PY_LARGEUR_BANDE 0.18
PY_NB_BANDES 4
PY_ANGLE_ORIENTATION 13
PY_TYPE_CULT 1
PY_FINESSE 1
PY_KEYKEY 10000
PY_cepage gre
SOILS_calc 35.8
PY ; xmlfile : sols.xml
[1] "R :"
[1] "xmlfile : R_related\\Rproject\\sols.xml"
[1] "name : calc"
[1] "value : 35.8"
[1] "val_id : "
SOILS_norg 0.15
PY ; xmlfile : sols.xml
[1] "R :"
[1] "xmlfile : R_related\\Rproject\\sols.xml"
[1] "name : norg"
[1] "value : 0.15"
[1] "val_id : "
SOILS_argi 22
PY ; xmlfile : sols.xml
[1] "R :"
[1] "xmlfile : R_related\\Rproject\\sols.xml"
[1] "name : argi"
[1] "value : 22"
[1] "val_id : "
SOILS_profhum 35
PY ; xmlfile : sols.xml
[1] "R :"
[1] "xmlfile : R_related\\Rproject\\sols.xml"
[1] "name : profhum"
[1] "value : 35"
[1] "val_id : "
SOILS_CsurNsol 10
PY ; xmlfile : sols.xml
[1] "R :"
[1] "xmlfile : R_related\\Rproject\\sols.xml"
[1] "name : CsurNsol"
[1] "value : 10"
[1] "val_id : "
SOILS_pH 8.2
PY ; xmlfile : sols.xml
[1] "R :"
[1] "xmlfile : R_rela

In [ ]:
os.path.join('R_related','Rproject','sols.xml')

'R_related\\Rproject\\sols.xml'

In [102]:
list(range(1, 5))

[1, 2, 3, 4]